In [2172]:
import pandas as pd, numpy as np, math
from json import loads, dumps
from tqdm import tqdm

e = math.exp(1)

In [2173]:
proprietes = {}
data_set = pd.DataFrame()
ansers = pd.Series()

In [2174]:
def get_proprietes():
    global proprietes
    with open("Network_Proprietes.json") as file:
        proprietes = file.read()
    proprietes = loads(proprietes)

In [2175]:
def get_dataset():
    global ansers
    global data_set
    data_set = pd.read_csv(proprietes["Data Set"] + ".csv")
    data_set = data_set.head(30)
    data_set.drop(columns= proprietes["Drop colunms"], inplace= True)
    ansers = data_set[proprietes["Output"]].copy()
    ansers = np.ceil(ansers.clip(0, 1))
    data_set.drop(columns= proprietes["Output"], inplace= True)
    data_set.reset_index(inplace= True, drop= True)
    data_set = data_set.transpose()
    data_set.reset_index(inplace= True, drop= True)

In [2176]:
class Layer:
    def __init__(self, num_inputs, num_neurons, hidden= True):
        self.weights = pd.DataFrame(np.random.randn(num_inputs, num_neurons))
        self.biases = pd.DataFrame(np.zeros((1, num_neurons)))
        self.hidden = hidden

    def forward(self, input, ansers= [], learn= False):
        self.output = (self.weights.transpose()).dot(input.transpose())
        self.output = self.output.transpose() + (self.biases.values.tolist()[0])
        if self.hidden:
            self.output.clip(0, inplace= True)
        else:
            self.output = (e ** ((self.output.transpose() - self.output.max(axis= 1)).transpose()))
            self.output = (self.output.transpose() / self.output.sum(axis= 1)).transpose()
            if learn:
                avg = self.output[pd.get_dummies(ansers.transpose())].transpose().max().mean()
                self.output = 1 - avg

    def update(self):
        self.weights += pd.DataFrame(np.random.randn(self.weights.shape[0], self.weights.shape[1]) * proprietes["Intensity"])
        self.biases += pd.DataFrame(np.random.randn(self.biases.shape[0], self.biases.shape[1]) * proprietes["Intensity"])

In [2177]:
layers = []
def init_layers():
    inputs = proprietes["Input"]
    for neuron_count, idx in zip(proprietes["Layers"], range(len(proprietes["Layers"]))):
        if not idx+1 == len(proprietes["Layers"]):
            layers.append(Layer(inputs, neuron_count))
            inputs = neuron_count
        else:
            layers.append(Layer(inputs, neuron_count, hidden= False))

In [2178]:
layers_save = []

def save_layers():
    global layers_save
    layers_save = layers.copy()

def load_layers():
    global layers
    layers = layers_save.copy()

In [2179]:
def run_batch(batch_num, batch_count, lowest_loss):
    X = data_set.transpose().loc[(data_set.transpose().index % batch_count) == batch_num].copy()
    y = ansers.loc[(ansers.index % batch_count) == batch_num].copy()
    X.reset_index(inplace= True, drop= True)
    y.reset_index(inplace= True, drop= True)
    output = []
    for layer in layers:
        idx = layers.index(layer)
        if idx == 0:
            layer.forward(X)
            output = layer.output
        elif not idx+1 == len(layers):
            layer.forward(output)
            output = layer.output
        else:
            layer.forward(output, ansers= y, learn= True)
            if layer.output > lowest_loss:
                layer.update()
                return None
            else:
                return layer.output

In [2180]:
def learn():
    generations = proprietes["Generations"]
    batch_count = math.floor(data_set.shape[1] / proprietes["Batch Size"])
    lowest_loss = 1
    pbar = tqdm(desc="Processing dataset", colour='#0997FF', total= generations * batch_count)
    for generation in range(generations):
        for batch in range(batch_count):
            pbar.desc = f"Processing dataset (Loss: {round(lowest_loss, 4)}, Gen: {generation+1}/{generations})"
            pbar.update(1)
            save_layers()
            loss = run_batch(batch, batch_count, lowest_loss)
            if not loss is None:
                lowest_loss = loss
                load_layers()

In [2181]:
get_proprietes()
get_dataset()
init_layers()
learn()

Processing dataset (Loss: 0.5, Gen: 100/100): 100%|██████████| 700/700 [00:07<00:00, 91.37it/s]
